In [0]:
from delta.tables import DeltaTable

# Define os caminhos de armazenamento no Data Lake
gold_path = "/Volumes/workspace/lhdw/gold/vendas_delta/fato_vendas"

# Executa vacuum para remover arquivos não utilizados com mais de 7 dias
# Periodo (7 dias = 168 horas)
delta_table = DeltaTable.forPath(spark, gold_path)
delta_table.vacuum(168)


In [0]:
from delta.tables import *
# Define os caminhos de armazenamento no Data Lake
gold_path = "/Volumes/workspace/lhdw/gold/vendas_delta/fato_vendas"
# Otimiza a tabela combinando arquivos pequenos em arquivos maiores
delta_table = DeltaTable.forPath(spark, gold_path)
delta_table.optimize().executeCompaction()


DataFrame[path: string, metrics: struct<autoCompactParallelismStats:void,clusteringMetrics:void,clusteringStats:void,deletionVectorStats:struct<numDeletionVectorRowsRemoved:bigint,numDeletionVectorsRemoved:bigint>,endTimeMs:bigint,filesAdded:struct<avg:double,max:void,min:void,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<avg:double,max:void,min:void,totalFiles:bigint,totalSize:bigint>,numBatches:bigint,numBins:bigint,numBytesSkippedToReduceWriteAmplification:bigint,numFilesAdded:bigint,numFilesRemoved:bigint,numFilesSkippedToReduceWriteAmplification:bigint,numTableColumns:bigint,numTableColumnsWithStats:bigint,partitionsOptimized:bigint,preserveInsertionOrder:boolean,recompressionCodec:void,skippedArchivedFiles:bigint,startTimeMs:bigint,totalClusterParallelism:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,totalScheduledTasks:bigint,totalTaskExecutionTimeMs:bigint,zOrderStats:void>]

In [0]:

from delta.tables import DeltaTable, DeltaOptimizeBuilder
# Executa optimize com Z-Ordering na coluna "DataVenda"

# Load the Delta table
delta_table = DeltaTable.forPath(spark, gold_path)

# Execute Z-Ordering optimization on the column "DataVenda"
#delta_table.optimize().#executeZOrderBy("DataVenda").#execute()

spark.sql(f"""OPTIMIZE delta.`{gold_path}` ZORDER BY (DataVenda)""")


DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,clusteringStats:struct<inputZCubeFiles:struct<numFiles:bigint,size:bigint>,inputOtherFiles:struct<numFiles:bigint,size:bigint>,inputNumZCubes:bigint,mergedFiles:struct<numFiles:bigint,size:bigint>,numOutputZCubes:bigint>,numBins:bigint,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,

In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import lit, max, current_timestamp

# Carregar a tabela Delta
delta_table = DeltaTable.forPath(spark, "/Volumes/workspace//lhdw/gold/vendas_delta/dim_fabricante")

# Calcular o próximo valor de SK_Fabricante
next_sk = delta_table.toDF().select(max("sk_fabricante")).collect()[0][0] + 1

# Criar uma nova linha para ser inserida
new_row = spark.createDataFrame([
    (8, "Novo Fabricante", next_sk)  # Supondo que 8 seja um IDFabricante de exemplo
], ["IDFabricante", "Fabricante", "sk_fabricante"])

# Adicionar a coluna DataAtualizacao com o tipo correto
new_row = new_row.withColumn("data_atualizacao", current_timestamp())

# Executar a operação de inserção
delta_table.alias("target").merge(
    new_row.alias("source"),
    "target.IDFabricante = source.IDFabricante"
).whenNotMatchedInsertAll().execute()


DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

###Exemplo de Update

In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import col, current_timestamp

# Exemplo de update
delta_table = DeltaTable.forPath(spark, "/Volumes/workspace//lhdw/gold/vendas_delta/dim_fabricante")
delta_table.update(
    condition = col("IDFabricante") == 7,  # Condição de update
    set = { 
        "Fabricante": "'VanArsdel Inc.'",
        "data_atualizacao": current_timestamp()
    }  # Atualização de valor
)

DataFrame[num_affected_rows: bigint]

###Exemplo de Delete

In [0]:
# Exemplo de delete
delta_table.delete(condition = col("IDFabricante") == 8)



DataFrame[num_affected_rows: bigint]

###Exemplo de UPSERT

In [0]:
from pyspark.sql import SparkSession
from delta.tables import DeltaTable
from pyspark.sql.functions import monotonically_increasing_id, current_timestamp

# Carregue o DataFrame de origem (novos dados)
df_silver = spark.read.format("parquet").load("/Volumes/workspace/lhdw/silver/vendas/")

#Nome tabela destino
tb_destino = "dim_fabricante"

# Extrair produtos únicos para a dimensão Fabricante    
dim_fabricante_df = df_silver.select(
    "IDFabricante", "Fabricante").dropDuplicates()

# Adicionar chave substituta (surrogate keys)
dim_fabricante_df = dim_fabricante_df.withColumn("sk_fabricante", monotonically_increasing_id()+1)
# Carregue o DataFrame de destino (tabela existente)
df_target = DeltaTable.forPath(spark, "/Volumes/workspace/lhdw/gold/vendas_delta/dim_fabricante")

# Realize a operação de merge
df_target.alias("target").merge(
    dim_fabricante_df.alias("source"),
    "target.IDFabricante = source.IDFabricante"
).whenMatchedUpdate(set={
    "Fabricante": "source.Fabricante",
    "SK_Fabricante": "source.SK_Fabricante",
    "data_atualizacao": current_timestamp()
}).whenNotMatchedInsert(values={
    "Fabricante": "source.Fabricante",
    "IDFabricante": "source.IDFabricante",
    "SK_Fabricante": "source.SK_Fabricante",
    "data_atualizacao": current_timestamp()
}).execute()


DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import *

# Visualizar o histórico da tabela Delta
history_df = DeltaTable.forPath(spark, "/Volumes/workspace/lhdw/gold/vendas_delta/dim_fabricante").history()

display(history_df)

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
9,2025-08-27T23:26:48.000Z,8795980509812597,77andrerosa@gmail.com,MERGE,"Map(predicate -> [""(IDFabricante#17273 = IDFabricante#17284)""], clusterBy -> [], matchedPredicates -> [{""actionType"":""update""}], statsOnLoad -> false, notMatchedBySourcePredicates -> [], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(3187551953004534),0827-223943-pqpmleh1-v2n,8,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 1, numTargetBytesAdded -> 1336, numTargetBytesRemoved -> 1373, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 1, executionTimeMs -> 2562, materializeSourceTimeMs -> 524, numTargetRowsInserted -> 0, numTargetRowsMatchedDeleted -> 0, numTargetDeletionVectorsUpdated -> 0, scanTimeMs -> 1085, numTargetRowsUpdated -> 1, numOutputRows -> 1, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 1, numTargetFilesRemoved -> 1, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 876)",null,Databricks-Runtime/17.1.x-aarch64-photon-scala2.13
8,2025-08-27T23:26:07.000Z,8795980509812597,77andrerosa@gmail.com,DELETE,"Map(predicate -> [""(IDFabricante#17127 = 8)""])",null,List(3187551953004534),0827-223943-pqpmleh1-v2n,7,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 1372, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 913, numDeletionVectorsUpdated -> 0, numDeletedRows -> 1, scanTimeMs -> 855, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 52)",null,Databricks-Runtime/17.1.x-aarch64-photon-scala2.13
7,2025-08-27T23:25:55.000Z,8795980509812597,77andrerosa@gmail.com,UPDATE,"Map(predicate -> [""(IDFabricante#16847 = 7)""])",null,List(3187551953004534),0827-223943-pqpmleh1-v2n,6,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 1090, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 2345, numDeletionVectorsUpdated -> 0, scanTimeMs -> 1425, numAddedFiles -> 1, numUpdatedRows -> 1, numAddedBytes -> 1373, rewriteTimeMs -> 907)",null,Databricks-Runtime/17.1.x-aarch64-photon-scala2.13
6,2025-08-27T23:25:30.000Z,8795980509812597,77andrerosa@gmail.com,MERGE,"Map(predicate -> [""(cast(IDFabricante#16698 as bigint) = IDFabricante#16714L)""], clusterBy -> [], matchedPredicates -> [], statsOnLoad -> false, notMatchedBySourcePredicates -> [], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(3187551953004534),0827-223943-pqpmleh1-v2n,5,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 1, numTargetBytesAdded -> 1372, numTargetBytesRemoved -> 0, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 0, executionTimeMs -> 1112, materializeSourceTimeMs -> 8, numTargetRowsInserted -> 1, numTargetRowsMatchedDeleted -> 0, numTargetDeletionVectorsUpdated -> 0, scanTimeMs -> 0, numTargetRowsUpdated -> 0, numOutputRows -> 1, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 1, numTargetFilesRemoved -> 0, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 1028)",null,Databricks-Runtime/17.1.x-aarch64-photon-scala2.13
5,2025-08-27T22:32:22.000Z,8795980509812597,77andrerosa@gmail.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> false, partitionBy -> [])",null,List(3187551953004531),0827-213204-n5txtbgw-v2n,4,WriteSerializable,false,"Map(numFiles -> 1, numRemovedFiles -> 0, numRemovedBytes -> 0, numOutputRows -> 1, numOutputBytes -> 1090)",null,Databricks-Runtime/17.1.x-photon-scala2.13
4,2025-08-27T18:59:27.000Z,8795980509812597,77andrerosa@gmail.

### Acessar versões antigas (Time Travel):

In [0]:
# Acessar a versão 5 da tabela
df = spark.read.format("delta").option("versionAsOf", 9).load("/Volumes/workspace/lhdw/gold/vendas_delta/dim_fabricante")
display(df)


IDFabricante,Fabricante,sk_fabricante,data_atualizacao
7,VanArsdel,1,2025-08-27T23:26:44.916Z


In [0]:
# Restaurando versão antiga da tabela
from delta.tables import DeltaTable

# Inicializar DeltaTable
delta_table_path = "dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/dim_fabricante"
delta_table = DeltaTable.forPath(spark, delta_table_path)

# Restaurar a tabela para a versão 
delta_table.restoreToVersion(5)

# Apresentar a tabela
display(spark.read.format("delta").load(delta_table_path))


IDFabricante,Fabricante,sk_fabricante,data_atualizacao
7,VanArsdel,1,null


In [0]:
# Reparticionando a tabela fato_vendas por ano e mês e salvando o resultado
# Agrupando os arquivos pequenos em arquivos maiores
df = spark.read.format("delta").load("/Volumes/workspace/lhdw/gold/vendas_delta/fato_vendas")
df.repartition(2).write.format("delta").mode("overwrite").save("/Volumes/workspace/lhdw/gold/vendas_delta/fato_vendas_repart")

In [0]:
# Lê a tabela Delta
df_geo = spark.read.format("delta").load("/Volumes/workspace/lhdw/gold/vendas_delta/dim_geografia")

# Estima o número de partições iniciais pelo número de arquivos
num_particoes_iniciais = len(set(df_geo.inputFiles()))
print(f"Número de partições iniciais (aproximado): {num_particoes_iniciais}")

# Reparticiona para 2 partições
df_reparticionado = df_geo.repartition(2)

# Persiste os dados reparticionados em uma nova tabela Delta
df_reparticionado.write.format("delta")\
    .option("overwriteSchema", "true")\
    .mode("overwrite")\
    .save("/Volumes/workspace/lhdw/gold/vendas_delta/geo_repart")

# Lê novamente e estima o número de partições após o reparticionamento
df_check = spark.read.format("delta").load("/Volumes/workspace/lhdw/gold/vendas_delta/geo_repart")
num_particoes_finais = len(set(df_check.inputFiles()))
print(f"Número de partições após reparticionamento (aproximado): {num_particoes_finais}")

Número de partições iniciais (aproximado): 1
Número de partições após reparticionamento (aproximado): 2


In [0]:
# Reparticionando com coluna especifica
# Lê a tabela Delta
df_geo = spark.read.format("delta").load("/Volumes/workspace/lhdw/gold/vendas_delta/dim_geografia")

# Reparticiona os dados pela coluna "Regiao"
df_reparticionado_geo = df_geo.repartition(10, "Regiao")

# Persiste os dados em uma tabela Delta particionada por "Regiao"
df_reparticionado_geo.write.partitionBy("Regiao").format("delta").option("overwriteSchema", "true").mode("overwrite").save("/Volumes/workspace/lhdw/gold/vendas_delta/geo_regiao")

# Lê a tabela Delta reparticionada
df_check = spark.read.format("delta").load("/Volumes/workspace/lhdw/gold/vendas_delta/geo_regiao")

# Estima o número de partições pelo número de arquivos únicos
num_particoes = len(set(df_check.inputFiles()))
print(f"Número de partições após reparticionar pela coluna Regiao (aproximado): {num_particoes}")

Número de partições após reparticionar pela coluna Regiao (aproximado): 3


In [0]:
# Usando coalesce para reduzir as partições para 5
df_coalesced = df_geo.repartition(100).coalesce(5)

# Persiste os dados em uma tabela Delta
df_coalesced.write.format("delta").option("overwriteSchema", "true").mode("overwrite").save("/Volumes/workspace/lhdw/gold/vendas_delta/geo_coalesce")

# Lê a tabela Delta reparticionada
df_check = spark.read.format("delta").load("/Volumes/workspace/lhdw/gold/vendas_delta/geo_coalesce")

# Estima o número de partições pelo número de arquivos únicos
num_particoes = len(set(df_check.inputFiles()))
print(f"Número de partições após coalesce (aproximado): {num_particoes}")

Número de partições após coalesce (aproximado): 5


###Evidências de repartição/Compactação

In [0]:
%fs ls /Volumes/workspace/lhdw/gold/vendas_delta/

path,name,size,modificationTime
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/dim_categoria/,dim_categoria/,0,1756337837570
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/dim_cliente/,dim_cliente/,0,1756337837571
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/dim_fabricante/,dim_fabricante/,0,1756337837571
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/dim_geografia/,dim_geografia/,0,1756337837571
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/dim_produto/,dim_produto/,0,1756337837571
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/dim_segmento/,dim_segmento/,0,1756337837571
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/fato_vendas/,fato_vendas/,0,1756337837571
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/fato_vendas_repart/,fato_vendas_repart/,0,1756337837571
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/geo_coalesce/,geo_coalesce/,0,1756337837571
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/geo_coaslece/,geo_coaslece/,0,1756337837571


In [0]:
%fs ls /Volumes/workspace/lhdw/gold/vendas_delta/fato_vendas_repart/

path,name,size,modificationTime
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/fato_vendas_repart/_delta_log/,_delta_log/,0,1756337858657
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/fato_vendas_repart/part-00000-d4b81e20-6c69-4f3e-af41-103ee05d14f9.c000.snappy.parquet,part-00000-d4b81e20-6c69-4f3e-af41-103ee05d14f9.c000.snappy.parquet,382178,1756337399000
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/fato_vendas_repart/part-00001-d848caa6-c872-428f-ad48-a34c19725bef.c000.snappy.parquet,part-00001-d848caa6-c872-428f-ad48-a34c19725bef.c000.snappy.parquet,382632,1756337399000


In [0]:
%fs ls /Volumes/workspace/lhdw/gold/vendas_delta/geo_repart/

path,name,size,modificationTime
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/geo_repart/_delta_log/,_delta_log/,0,1756337876886
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/geo_repart/part-00000-ab25a141-a7d3-4276-8d42-b7cd3d3cb6cd.c000.snappy.parquet,part-00000-ab25a141-a7d3-4276-8d42-b7cd3d3cb6cd.c000.snappy.parquet,176729,1756337546000
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/geo_repart/part-00001-37b890c4-4a04-499c-8c98-da5a43b63b2e.c000.snappy.parquet,part-00001-37b890c4-4a04-499c-8c98-da5a43b63b2e.c000.snappy.parquet,177228,1756337546000


In [0]:
%fs ls /Volumes/workspace//lhdw/gold/vendas_delta/geo_regiao/

path,name,size,modificationTime
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/geo_regiao/Regiao=Central/,Regiao=Central/,0,1756337888605
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/geo_regiao/Regiao=East/,Regiao=East/,0,1756337888605
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/geo_regiao/Regiao=West/,Regiao=West/,0,1756337888605
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/geo_regiao/_delta_log/,_delta_log/,0,1756337888605


In [0]:
%fs ls /Volumes/workspace/lhdw/gold/vendas_delta/geo_coalesce/

path,name,size,modificationTime
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/geo_coalesce/_delta_log/,_delta_log/,0,1756337907056
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/geo_coalesce/part-00000-9620897a-51b5-48ab-a0f9-34c82db5b506.c000.snappy.parquet,part-00000-9620897a-51b5-48ab-a0f9-34c82db5b506.c000.snappy.parquet,71536,1756337811000
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/geo_coalesce/part-00000-b3a073a6-217d-4680-b0a2-18224c846950.c000.snappy.parquet,part-00000-b3a073a6-217d-4680-b0a2-18224c846950.c000.snappy.parquet,71536,1756337752000
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/geo_coalesce/part-00001-12f79acb-3b5e-4f4a-ac96-38a428387317.c000.snappy.parquet,part-00001-12f79acb-3b5e-4f4a-ac96-38a428387317.c000.snappy.parquet,72334,1756337811000
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/geo_coalesce/part-00001-d685828f-47d3-40d8-a65c-8c4e8b12a6ff.c000.snappy.parquet,part-00001-d685828f-47d3-40d8-a65c-8c4e8b12a6ff.c000.snappy.parquet,72334,1756337752000
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/geo_coalesce/part-00002-2493140a-128a-4267-97dc-e6a2fbabf801.c000.snappy.parquet,part-00002-2493140a-128a-4267-97dc-e6a2fbabf801.c000.snappy.parquet,72920,1756337752000
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/geo_coalesce/part-00002-576a8ba6-ce60-45e3-8911-eed324452ec1.c000.snappy.parquet,part-00002-576a8ba6-ce60-45e3-8911-eed324452ec1.c000.snappy.parquet,72920,1756337811000
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/geo_coalesce/part-00003-4e394508-9242-41b8-a2a4-2aeb79516c5b.c000.snappy.parquet,part-00003-4e394508-9242-41b8-a2a4-2aeb79516c5b.c000.snappy.parquet,71617,1756337752000
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/geo_coalesce/part-00003-d75a324a-5f6d-4e62-a519-3f2fce95b59b.c000.snappy.parquet,part-00003-d75a324a-5f6d-4e62-a519-3f2fce95b59b.c000.snappy.parquet,71617,1756337811000
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/geo_coalesce/part-00004-dfee49d8-3d27-447c-9703-8c9baed775e5.c000.snappy.parquet,part-00004-dfee49d8-3d27-447c-9703-8c9baed775e5.c000.snappy.parquet,72127,1756337811000
